In [24]:
# Imports and global variables
import requests
import json
import pandas as pd
from dateutil import parser

BASE_URL = "http://localhost:8080/engine-rest"

In [42]:
# CLEANUP
# delete all process definitions and running instances
# ! we need to delete all the running instances first

def status_code_successful(status_code: int):
    return str(status_code)[0] == '2'


def delete_all_data(target: str):
    POSSIBLE_TARGETS = ["process-instance", "process-definition", "deployment", "decision-definition"]
    if target not in POSSIBLE_TARGETS:
        raise Exception(str(target) + "not a valid data deletion target")
        
    get_response = requests.get(BASE_URL + "/" + target)
    assert(status_code_successful(get_response.status_code))
    #print(json.dumps(get_response.json(), indent=2))

    for elem in get_response.json():
        current_id = elem.get('id')
        del_response = requests.delete(BASE_URL + "/" + target + "/" + str(current_id))
        #print(del_response)
        assert(status_code_successful(del_response.status_code))

    
    get_response = requests.get(BASE_URL + "/" + target)
    assert(status_code_successful(get_response.status_code))
    assert(len(get_response.json()) == 0)   

    
for elem in ["process-instance", "process-definition", "deployment", "decision-definition"]:
    delete_all_data(elem)



In [43]:
# deploy ONE example process 
# TODO: extent for multiple

multipart_form_data = { 
    'deployment-name': (None, 'store'),
    'data': ('helicopter_fast_vA.bpmn', open('helicopter_fast_vA.bpmn', 'r')),
}

response = requests.post(BASE_URL + "/deployment/create", files=multipart_form_data)
assert(status_code_successful(response.status_code))

#print(json.dumps(response.json(), indent=2))
for elem in response.json().get('deployedProcessDefinitions'):
    new_process_id = elem

print(new_process_id)

Process_1cl94bg:1:77f1cbcf-845e-11ec-9c12-0242ac120003


In [44]:
from time import sleep
# start batch of instances
for x in range(1000):
    headers = {'Content-Type': 'application/json'}
    response = requests.post("http://localhost:8080/engine-rest/process-definition/" + str(new_process_id)+ "/start", headers=headers)
    #print(json.dumps(response.json(), indent=2))
    assert(status_code_successful(response.status_code))
    sleep(1)



In [45]:
# poll history service
# calculate durations of process instances by process definition
import datetime as dt

proc_def_response = requests.get(BASE_URL + "/history/process-definition/cleanable-process-instance-report")
assert(status_code_successful(proc_def_response.status_code))
#print(json.dumps(proc_def_response.json(), indent=2))

historic_df = pd.DataFrame(columns=['processDefinitionId', 'processDefinitionKey', 'processDefinitionName', 'Duration'])

for elem in proc_def_response.json():
    processDefinitionId = elem.get('processDefinitionId')
    processDefinitionKey = elem.get('processDefinitionKey')
    processDefinitionName = elem.get('processDefinitionName')
    params = {'processDefinitionId': str(processDefinitionId)}
    proc_inst_response = requests.get(BASE_URL + "/history/process-instance", params=params)
    assert(status_code_successful(proc_inst_response.status_code))
    for elem2 in proc_inst_response.json():
        end_datetime = parser.parse(elem2.get('endTime'))
        start_datetime = parser.parse(elem2.get('startTime'))
        duration = (end_datetime - start_datetime).total_seconds()
        historic_df.loc[len(historic_df)] = [processDefinitionId, processDefinitionKey, processDefinitionName, duration]
        
print(historic_df)
historic_df.to_csv('version_a_iteration=1000.csv', index=False)
# Calculate mean of duration of all instances
print("Mean: " + str((historic_df.Duration.mean())))

0.085
0.087
0.252
0.161
0.061
0.086
0.105
0.183
0.078
0.128
0.05
0.234
0.088
0.068
0.048
0.121
0.048
0.157
0.085
0.107
0.071
0.258
0.076
0.156
0.068
0.104
0.068
0.088
0.089
0.12
0.095
0.303
0.077
0.083
0.098
0.193
0.057
0.055
0.088
0.239
0.275
0.086
0.092
0.077
0.111
0.185
0.082
0.181
0.074
0.139
0.095
0.133
0.061
0.084
0.148
0.068
0.081
0.05
0.139
0.061
0.124
0.093
0.071
0.112
0.109
0.173
0.057
0.097
0.076
0.079
0.101
0.128
0.106
0.309
0.101
0.1
0.087
0.087
0.111
0.082
0.121
0.259
0.097
0.106
0.094
0.068
0.058
0.062
0.086
0.072
0.085
0.212
0.071
0.082
0.124
0.083
0.063
0.093
0.045
0.067
0.071
0.074
0.079
0.073
0.062
0.048
0.076
0.294
0.114
0.11
0.066
0.126
0.076
0.113
0.057
0.245
0.077
0.116
0.078
0.172
0.09
0.094
0.227
0.198
0.084
0.07
0.085
0.115
0.086
0.083
0.335
0.147
0.097
0.118
0.092
0.08
0.149
0.078
0.138
0.101
0.08
0.126
0.162
0.18
0.179
0.171
0.112
0.102
0.077
0.113
0.104
0.249
0.103
0.206
0.088
0.303
0.115
0.171
0.081
0.22
0.082
0.198
0.119
0.108
0.345
0.429
0.162
0.513
0.41

In [36]:
df = pd.read_csv('version_a_iteration=1000.csv')
min_duration = df['Duration'].min()
max_duration = df['Duration'].max()
print(f'Min value: {min_duration}')
print(f'Max value: {max_duration}')

df.plot.hist(columns='Duration')

Min value: 0 days 00:00:00.020000
Max value: 0 days 00:00:00.773000
0    0 days 00:00:00.042000
Name: Duration, dtype: object
